In [1]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from scipy.interpolate import UnivariateSpline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import math
from fbprophet import Prophet
import matplotlib.pyplot as plt

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd /content/drive/'My Drive'/'CS145_Kaggle'/
# %ls


In [4]:
def getStates():
    ds = pd.read_csv('data/test.csv')
    states = ds['Province_State'][:50].values   
    return states

In [5]:
def isDown(state, ds):
    x = [[i] for i in range(142)]
    
    confirmed = ds['Confirmed'].values
    deaths = ds['Deaths'].values
    
    smooth_confirmed = savgol_filter(confirmed.reshape(-1), 31, 2)
    scale_smooth_confirmed = (smooth_confirmed - confirmed[0]) / confirmed[-14] * 128
    
    scale_deaths = deaths / deaths[-1] * 142
    deaths_spl = UnivariateSpline(x,scale_deaths,s=10,k=4)
    deaths_spl_1d = deaths_spl.derivative(n=1)
      
    if scale_smooth_confirmed[-1] - 142 < -7 and np.mean(deaths_spl_1d(x)[-28:-14]) < 1.5:
        return True

In [6]:
def distance(x, y):
    if y > x:
        return 0
    b = y + x
    i = b / 2
    return math.sqrt((x - i)**2 + (y - i)**2)

In [7]:
def all_distance(data):
    rtn = []
    for i in range(len(data)):
        rtn.append(distance(i, data[i]))
    return rtn

In [8]:
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

In [9]:
degree = 3
states = getStates()
start = 133
alpha = 6.5
window = 17

In [10]:
feature = 'Deaths'
total = 0
res = []

for i in range(len(states)):
    state = states[i]
    ds = pd.read_csv('data/train.csv')
    ds = ds[ds['Province_State'] == state]
    raw = ds[feature].values
    
    value = savgol_filter(raw.reshape(-1), window, degree) #smooth data
    
###===================================================================###   
    scale = raw / raw[-1] * 142
    distances = all_distance(scale)
    diff = sum(distances)
    max_point = distances.index(max(distances))       
    
    if scale[80] > 100: # Grow too fast, Saturated
        x = [[i] for i in range(142)]
        #print(scale[75], state)
        model = Pipeline([
            ("poly", PolynomialFeatures(degree=1)),
            ("lasso_reg", Ridge(alpha=0)) 
        ])
        model.fit(x[-10:], value[-10:])
        x_test = [[i + 142] for i in range(26)]
        y_hat = model.predict(x_test)
        y_hat = y_hat.reshape(-1,1) 
        
###===================================================================###
    else:  
        y = np.array([value[start:]]).reshape(-1, 1)
        x = [[i + start] for i in range(142-start)]
        
        diff = np.sum((raw[start:] - value[start:])**2 / raw[-1]**2)
        
        model = Ridge(alpha=alpha, fit_intercept=True)
        model.fit(x, y)
        x_test = [[i + 142] for i in range(26)]

        y_hat = model.predict(x_test)
        slope = (168 - 142) / (y_hat[-1] - y_hat[0]) * raw[-1] / 142
        
        if isDown(state, ds): # 
            if slope < 1.6:
                sub = [[np.log(1 + i) ** 2] for i in range(26)]
                y_hat = y_hat - sub

        
        else:
            if slope < 0 : # impossible slope
                y = np.array([value[50:]]).reshape(-1, 1)
                x = [[i + 50] for i in range(142 - 50)]

                model = Pipeline([
                    ("poly", PolynomialFeatures(degree=2)),
                    ("lasso_reg", Ridge(alpha=0)) 
                ])
                model.fit(x, y)

                y_hat = model.predict(x_test)
# ###===================================================================###        
    res.append(y_hat)

In [11]:
rerange = []
for i in range(len(res[0])):
    for j in range(len(res)):
        rerange.append(res[j][i])
#print(rerange)
Death_df = pd.DataFrame(rerange)
Death_df.to_csv("output_round1/{}.csv".format(feature),index=False,sep=',')

In [12]:
degree = 3
states = getStates()
start = 137
alpha = .5
window = 15
states2idx = {}
for i in range(len(states)):
    states2idx[states[i]] = i

In [13]:
feature = 'Confirmed'
res = []
total = 0

for i in range(len(states)):
    state = states[i]
    ds = pd.read_csv('data/train.csv')
    ds = ds[ds['Province_State'] == state]
    raw = ds[feature].values
    value = savgol_filter(raw.reshape(-1), window, degree) #smooth data
    
###===================================================================###   
    scale = raw / raw[-1] * 142
    distances = all_distance(scale)
    diff = sum(distances[-30:])
    max_point = distances.index(max(distances))       

    if scale[80] > 100: # Grow too fast, Saturated
        x = [[i] for i in range(142)]
        model = Pipeline([
            ("poly", PolynomialFeatures(degree=1)),
            ("lasso_reg", Ridge(alpha=0)) 
        ])
        model.fit(x[-10:], value[-10:])
        x_test = [[i + 142] for i in range(26)]
        y_hat = model.predict(x_test)
        y_hat = y_hat.reshape(-1,1)
###===================================================================###
    else:
        y = np.array([value[start:]]).reshape(-1, 1)
        x = [[i + start] for i in range(142-start)]
        model = Ridge(alpha=alpha)
        model.fit(x, y)
        x_test = [[i + 142] for i in range(26)]
        y_hat = model.predict(x_test)
        slope = (168 - 142) / (y_hat[-1] - y_hat[0]) * raw[-1] / 142
###===================================================================###
        move_in = pd.read_csv('data/move_in_data.csv')
        move_out = pd.read_csv('data/move_out_data.csv')

        idx = states2idx[state]

        move_in = move_in.iloc[[idx]].values[0][2:]
        move_out = move_out.iloc[[idx]].values[0][2:]

        smooth_move_in = savgol_filter(move_in.reshape(-1), 11, 3)
        smooth_move_out = savgol_filter(move_out.reshape(-1), 11, 3)

        move_diff = move_in - move_out
###===================================================================###
        all_neg = True
        for i in move_diff[-30:]:
            if i > 0:
                all_neg = False
                break
 
        if all_neg and (np.mean(move_diff[-14:]) - np.mean(move_diff[-28:-14]) < 0):# tend to slow down
            move_slope = 14 / np.mean(move_diff[-14:]) - np.mean(move_diff[-28:-14]) / np.mean(move_diff[-14:-7])
            y_hat = (y_hat - y_hat[0]) * (-move_slope) + y_hat[0] # 1 - (1 - |move_slope|)
###===================================================================###        
        all_pos = True    
        for i in move_diff[-30:]:
            if i < 0:
                all_pos = False
                break
 
        if all_pos and (np.mean(move_diff[-14:]) - np.mean(move_diff[-28:-14]) > 0):# tend to speed up
            move_slope = 14 / np.mean(move_diff[-14:]) - np.mean(move_diff[-28:-14]) / np.mean(move_diff[-14:-7])
            y_hat = (y_hat - y_hat[0]) * (2 + move_slope) + y_hat[0] # 1 + (1 - |move_slope|)          
###===================================================================###
        std_move_diff = standardization(move_diff)
        fst_mean = np.mean(std_move_diff[-28:-21])
        snd_mean = np.mean(std_move_diff[-14:-7])
        if abs(fst_mean - snd_mean) > 2: # detect sudden change of mobilility
            value = savgol_filter(value.reshape(-1), window, degree)
            y = np.array([value[start - 7:]]).reshape(-1, 1)
            x = [[i + start - 7] for i in range(142 - start + 7)] # double training set
            model = Ridge(alpha=alpha)
            model.fit(x, y)
            x_test = [[i + 142] for i in range(26)]
            y_hat = model.predict(x_test)    
###===================================================================###        
        state_data = ds[-21:]
        df_date = state_data['Date'].to_frame()
        df_y = state_data[feature].to_frame()
        df = pd.concat([df_y, df_date], axis=1)
        df.columns = ['y', 'ds']
        m = Prophet()
        m.fit(df)
        future = m.make_future_dataframe(periods=26)
        forecast = m.predict(future)
        sesonality = forecast['weekly'].values # extract seasonality from prophet
        sesonality = sesonality
        sesonality = sesonality[-26:].reshape(-1, 1)
        y_hat = y_hat + sesonality # apply seasonality to our result
###===================================================================###        
    res.append(y_hat)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [14]:
rerange = []
for i in range(len(res[0])):
    for j in range(len(res)):
        rerange.append(res[j][i])
#print(rerange)
Confirm_df = pd.DataFrame(rerange)
Confirm_df.to_csv("output_round1/{}.csv".format(feature),index=False,sep=',')

In [15]:
submission_df = pd.DataFrame({"ForecastID" : np.arange(26*50)})
submission_df = pd.concat([submission_df, Confirm_df, Death_df], axis = 1)
submission_df.columns = ["ForecastID", "Confirmed", "Deaths"]

In [16]:
submission_df

,ForecastID,Confirmed,Deaths
0,0,126768.444529,2190.364297
1,1,5307.139062,38.397745
2,2,202404.481222,5076.520324
3,3,61518.378386,805.358833
4,4,716449.952860,13122.890354
...,...,...,...
1295,1295,145875.347399,2936.681731
1296,1296,86965.981078,2062.577514
1297,1297,14024.443660,333.989340
1298,1298,92791.615774,1249.219263


In [17]:
formatted_df = submission_df.copy()
formatted_df.index = submission_df['ForecastID']
formatted_df.drop(columns='ForecastID', inplace=True)
print(formatted_df)
formatted_df.to_csv('round1.csv')

                Confirmed        Deaths
ForecastID                             
0           126768.444529   2190.364297
1             5307.139062     38.397745
2           202404.481222   5076.520324
3            61518.378386    805.358833
4           716449.952860  13122.890354
...                   ...           ...
1295        145875.347399   2936.681731
1296         86965.981078   2062.577514
1297         14024.443660    333.989340
1298         92791.615774   1249.219263
1299          4674.081091     51.060289

[1300 rows x 2 columns]
